In [ ]:
# import and run the poisson1 script
import poisson1
import eztfem as ezt

results = poisson1.main()

max_diff = results[0]
mesh     = results[1]
problem  = results[2]
u        = results[3]
derivs   = results[4]

In [ ]:
# plot the mesh including curve numbers and point numbers
ezt.plot_points_curves(mesh)

In [ ]:
# create a PyVista mesh for visualizing results
mesh_pv = ezt.generate_pyvista_mesh(mesh)

In [ ]:

# plot the mesh in Pyvista
ezt.plot_mesh_pv(mesh_pv, show_edges=True, window_size=(800, 400))

In [ ]:
# plot the solution over a line
points = [[0.0, 0.3, 0.0], [1.0, 0.3, 0.0]]
data_sample = ezt.plot_sol_over_line(mesh_pv, problem, u, points, npoints=200) #, plot_mesh=True)

In [ ]:
# plot the solution field
ezt.plot_sol(mesh_pv, problem, u, show_scalar_bar=True, n_colors=16,
             window_size=(800, 400))